In [1]:
import numpy as np#
import matplotlib.pyplot as plt
import torch 
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms

In [2]:
transform = transforms.Compose([transforms.Resize((28,28)),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5,), (0.5,))
                               ])
training_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
validation_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [3]:
training_loader = torch.utils.data.DataLoader(training_dataset, batch_size = 100, shuffle = True)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size =100 , shuffle = True)

In [4]:
type(training_dataset)

image, label  = training_dataset[0]
print(image.shape)
# image  = image.numpy().squeeze()
# image = image*0.5 + 0.5
# plt.imshow(image, cmap='gray')
# plt.show()



torch.Size([1, 28, 28])


In [5]:
class Classifier(nn.Module):
    def __init__(self, num_in, H1 ,H2, num_out):
        super().__init__()
        self.layer1 = nn.Linear(num_in,H1)
        self.layer2 = nn.Linear(H1,H2)
        self.layer3 = nn.Linear(H2,num_out)
    def forward(self,x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        # i assume the soft max should be applied here !? (becaus the value won't be between 0-1)
        x = self.layer3(x)
        return x


In [6]:
model = Classifier(784, 125, 125, 10)
model.parameters

<bound method Module.parameters of Classifier(
  (layer1): Linear(in_features=784, out_features=125, bias=True)
  (layer2): Linear(in_features=125, out_features=125, bias=True)
  (layer3): Linear(in_features=125, out_features=10, bias=True)
)>

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [8]:
# When reshaping a tensor, the new shape must contain the same total number 
#of elements as the old shape. For example, if the original shape was [100, 1, 28, 28], 
#there are a total of 100 * 1 * 28 * 28 = 78400 elements.
# If you want to reshape this tensor to have a shape of [100, something],
#then something must be 78400 / 100 = 784 in order for the total number of elements
#to remain the same. You could calculate this yourself and use view(100, 784),
#but PyTorch allows you to just write view(100, -1), and it will automatically calculate 
#that the size of the second dimension should be 784.

In [17]:
epochs = 2
loss_history = []
correct_predection_history = []

for e  in  range(epochs):
    running_loss= 0.0
    running_corrects = 0.0
    val_running_loss= 0.0
    val_running_corrects = 0.0
    for batch, counter in zip(training_loader, range(3)):
        examples , labels = batch
        examples = examples.view(examples.shape[0],-1)
        y_dach = model(examples)
        loss = criterion(y_dach,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, preds = torch.max(y_dach, 1)
        running_loss += loss.item()
        running_corrects += torch.sum(preds == labels)
    else: 
        with torch.no_grad():
            for val_batch , counter in zip(validation_loader, range(3)):
                val_examples , val_labels = val_batch
                val_examples = val_examples.view(val_examples.shape[0],-1)
                val_y = model(val_examples)
                loss = criterion(val_y, val_labels)
                _,val_preds = torch.max(val_y, 1)
                val_running_loss += loss.item()
                val_running_corrects += torch.sum(val_preds == val_labels)
                print('-------------')
                print(val_running_loss)
                print(val_running_corrects)
                
    
                
    
    



-------------
2.071519613265991
tensor(57.)
-------------
4.115206718444824
tensor(125.)
-------------
6.148845911026001
tensor(188.)
-------------
2.0217666625976562
tensor(61.)
-------------
4.059501647949219
tensor(111.)
-------------
6.101106882095337
tensor(169.)
